In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import sys
import time
from dataclasses import dataclass
from enum import Enum

from autoscript_sdb_microscope_client.structures import AdornedImage, StagePosition, ManipulatorPosition
from liftout.fibsem.sample import Sample, Lamella, AutoLiftoutState



### New Sample Data Structure
Prototyping for the new sample dataclass, and filesystem structures


In [125]:
import random
import os
from pprint import pprint

from liftout.fibsem.sample import create_experiment, load_experiment, get_current_lamella
from pathlib import Path

sample_path = "test_sample"
sample = create_experiment(sample_path)

for i in range(1, random.randint(3, 5)):

    lamella = Lamella(path=sample.path, number=i)
    lamella.lamella_coordinates = StagePosition(i, i, i, i, i)
    lamella.landing_coordinates = StagePosition(i, i, i, i, i)

    for j in range(random.randint(1, 6)):
        lamella.history.append(lamella.current_state)

    sample.positions[lamella._number] = lamella


pprint(sample)
pprint(sample.positions)

# save to file
sample.save()

# load from file
sample2 = load_experiment(sample_path)

# validation
assert sample.path == sample2.path
for i in sample.positions:
    assert sample.path[i] == sample2.path[i]
    assert sample.positions[i].lamella_coordinates == sample2.positions[i].lamella_coordinates
    assert sample.positions[i].landing_coordinates == sample2.positions[i].landing_coordinates
    assert sample.positions[i]._id == sample2.positions[i]._id
    assert sample.positions[i]._petname == sample2.positions[i]._petname
    assert sample.positions[i].current_state == sample2.positions[i].current_state
    assert sample.positions[i].history == sample2.positions[i].history
    print(f"lamella {i} passed.")


# manipulations
current_no = 1
current_lamella = get_current_lamella(sample, current_no)

# updates to dictionarys follow reference, not sure about others

# sample.save()


Sample: 
        Path: test_sample
        State: None
        Lamella: 3
        
{1: 
        Lamella 1 (01-daring-swine). 
        Lamella Coordinates: StagePosition(x=1, y=1, z=1, t=1, r=1), 
        Landing Coordinates: StagePosition(x=1, y=1, z=1, t=1, r=1), 
        Current Stage: AutoLiftoutStage.Initialisation,
        Last Completed: AutoLiftoutStage.Setup
        History: 6 stages completed (['Initialisation', 'Initialisation', 'Initialisation', 'Initialisation', 'Initialisation', 'Initialisation']).
        ,
 2: 
        Lamella 2 (02-trusty-bison). 
        Lamella Coordinates: StagePosition(x=2, y=2, z=2, t=2, r=2), 
        Landing Coordinates: StagePosition(x=2, y=2, z=2, t=2, r=2), 
        Current Stage: AutoLiftoutStage.Initialisation,
        Last Completed: AutoLiftoutStage.Setup
        History: 5 stages completed (['Initialisation', 'Initialisation', 'Initialisation', 'Initialisation', 'Initialisation']).
        ,
 3: 
        Lamella 3 (03-clear-shrew). 
     

### New Protocol
- Splitting the protocol into separate files. 
    - load into a single settings, dictionary
    - save protocol into individual files 
    - load protocol individually
    - possibly change to loading experiments

In [143]:
base_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout"

from pprint import pprint
from liftout.utils import load_config_v2

system_fname = os.path.join(base_path, "config", "system.yaml")
calibration_fname = os.path.join(base_path, "config", "calibration.yaml")
protocol_fname = os.path.join(base_path, "protocol", "protocol.yaml")

settings = load_config_v2(
    system_fname, 
    calibration_fname, 
    protocol_fname
)

pprint(settings)


{'calibration': {'calibration': {'drift_correction_hfw_highres': 8e-05,
                                 'eucentric_height_eb': 0.004,
                                 'eucentric_height_ib': 0.0165,
                                 'eucentric_height_tolerance': 0.0005,
                                 'eucentric_hfw_highres': 0.0002,
                                 'eucentric_hfw_lowres': 0.0009,
                                 'eucentric_hfw_midres': 0.0004,
                                 'max_eb_hfw': 0.0027,
                                 'max_ib_hfw': 0.0009,
                                 'max_working_distance_eb': 0.006,
                                 'stage_height_limit': 0.0037},
                 'gamma': {'correction': 1.0,
                           'max_gamma': 1.8,
                           'min_gamma': 0.15,
                           'scale_factor': 0.01,
                           'threshold': 46.0},
                 'imaging': {'autocontrast': 1.0,
          

In [145]:


def update_protocol(settings: dict, step: str, fname: Path) -> None:

    print("step: ", step)
    pprint(settings["protocol"][step])
    print("fname: ", fname)
    print("-"*50)


# from liftout.fibsem.milling import get_milling_protocol_stages
# protocol_stages = get_milling_protocol_stages(settings, "lamella")

# pprint(protocol_stages)

for k in settings["protocol"].keys():
    update_protocol(settings, k, None)


step:  lamella
{'lamella_height': 6e-06,
 'lamella_width': 1.2e-05,
 'milling_current': 7.6e-09,
 'milling_depth': 1.5e-05,
 'protocol_stages': [{'milling_current': 7.6e-09,
                      'offset': 2e-06,
                      'trench_height': 1e-05},
                     {'milling_current': 7.6e-09,
                      'offset': 0.0,
                      'trench_height': 3e-06}],
 'size_ratio': 1.0}
fname:  None
--------------------------------------------------
step:  jcut
{'extra_bit': 8e-06,
 'height': 5e-06,
 'jcut_angle': 6.0,
 'length': 1.2e-05,
 'milling_current': 2e-09,
 'milling_depth': 3e-06,
 'trench_thickness': 1e-06}
fname:  None
--------------------------------------------------
step:  weld
{'depth': 2e-07, 'height': 5e-06, 'rotation': 0.0, 'width': 3e-06}
fname:  None
--------------------------------------------------
step:  sharpen
{'bias': 2.5e-07,
 'depth': 8e-06,
 'height': 2.5e-06,
 'hfw': 0.00015,
 'milling_current': 7.6e-09,
 'needle_angle': 40.0,
 'ti

: 

: 

### Update Image Settings
new functional image settings update

In [36]:
# update_image_settings
%load_ext autoreload
%autoreload 2

import datetime
import time
from liftout.fibsem.acquire import GammaSettings, ImageSettings, BeamType
from liftout import utils
from pprint import pprint
import logging


def update_image_settings_v3(
    settings: dict,
    resolution=None,
    dwell_time=None,
    hfw=None,
    autocontrast=None,
    beam_type=None,
    gamma=None,
    save=None,
    label=None,
    path=None,
):
    """Update image settings. Uses default values if not supplied

    Args:
        settings (dict): the default settings dictionary
        resolution (str, optional): image resolution. Defaults to None.
        dwell_time (float, optional): image dwell time. Defaults to None.
        hfw (float, optional): image horizontal field width. Defaults to None.
        autocontrast (bool, optional): use autocontrast. Defaults to None.
        beam_type (BeamType, optional): beam type to image with (Electron, Ion). Defaults to None.
        gamma (GammaSettings, optional): gamma correction settings. Defaults to None.
        save (bool, optional): save the image. Defaults to None.
        label (str, optional): image filename . Defaults to None.
        save_path (Path, optional): directory to save image. Defaults to None.
    """
    gamma_settings = GammaSettings(
        enabled=settings["calibration"]["gamma"]["correction"],
        min_gamma=settings["calibration"]["gamma"]["min_gamma"],
        max_gamma=settings["calibration"]["gamma"]["max_gamma"],
        scale_factor=settings["calibration"]["gamma"]["scale_factor"],
        threshold=settings["calibration"]["gamma"]["threshold"],
    )

    # new image_settings
    image_settings = ImageSettings(
        resolution=settings["calibration"]["imaging"]["resolution"]
        if resolution is None
        else resolution,
        dwell_time=settings["calibration"]["imaging"]["dwell_time"]
        if dwell_time is None
        else dwell_time,
        hfw=settings["calibration"]["imaging"]["horizontal_field_width"]
        if hfw is None
        else hfw,
        autocontrast=settings["calibration"]["imaging"]["autocontrast"]
        if autocontrast is None
        else autocontrast,
        beam_type=BeamType.ELECTRON if beam_type is None else beam_type,
        gamma=gamma_settings if gamma is None else gamma,
        save=bool(settings["calibration"]["imaging"]["save"])
        if save is None else save,
        save_path="" if path is None else path,
        label=utils.current_timestamp()
        if label is None
        else label,
    )

    # TODO: the save path will be broken now...
    logging.debug(f"Image Settings: {image_settings}")

    return image_settings



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:

from liftout.fibsem.sample import load_experiment
sample_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\log\1245_2022-07-21.03-52PM"
sample = load_experiment(sample_path)
settings = utils.load_config_v2()


In [43]:
image_settings = update_image_settings_v3(settings, path=sample.path)

pprint(image_settings)
pprint(settings["calibration"]["imaging"])


image_settings = update_image_settings_v3(settings, path=image_settings.save_path)

pprint(image_settings)
pprint(settings["calibration"]["imaging"])




ImageSettings(resolution='1536x1024', dwell_time=1e-06, hfw=0.00015, autocontrast=1.0, beam_type=<BeamType.ELECTRON: 1>, save=False, label='2022-07-21.06-07-49PM', gamma=GammaSettings(enabled=1.0, min_gamma=0.15, max_gamma=1.8, scale_factor=0.01, threshold=46.0), save_path='c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\1245_2022-07-21.03-52PM\\1245_2022-07-21.03-52PM')
{'autocontrast': 1.0,
 'dwell_time': 1e-06,
 'horizontal_field_width': 0.00015,
 'imaging_current': 2e-11,
 'milling_current': 2e-09,
 'resolution': '1536x1024',
 'save': 0.0}
ImageSettings(resolution='1536x1024', dwell_time=1e-06, hfw=0.00015, autocontrast=1.0, beam_type=<BeamType.ELECTRON: 1>, save=False, label='2022-07-21.06-07-49PM', gamma=GammaSettings(enabled=1.0, min_gamma=0.15, max_gamma=1.8, scale_factor=0.01, threshold=46.0), save_path='c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\1245_2022-07-21.03-52PM\\1245_2022-07-21.03-52PM')
{'autocontrast': 1.0,
 'dwell_time': 1e-

Sample: 
        Path: c:\users\pcle0002\documents\repos\autoliftout\liftout\log\1245_2022-07-21.03-52PM\1245_2022-07-21.03-52PM
        State: None
        Lamella: 0
        